In [46]:
#!/usr/bin/env python3
"""Task 0. Initialize Generator"""
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [16]:
class Generator(nn.Module):
    """Generator for the GAN"""
    def __init__(self, input_size, hidden_size, output_size):
        """Constructor for the Generator
        Args:
            input: is the dimensionality of the noise vector
            hidden_size: is the number of hidden units in the GAN
            output_size: is the dimensionality of the generated sample"""
        super(Generator, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.main = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size),
            nn.Tanh(),
            nn.Linear(self.hidden_size, self.output_size),
            nn.Tanh(),
            nn.Linear(self.output_size, self.output_size)
        )

    def forward(self, x):
        """Defines the forward pass for the Generator"""
        return self.main(x)

In [21]:
"""Task 1. Create Discriminator"""


class Discriminator(nn.Module):
    """Discriminator for the GAN"""
    def __init__(self, input_size, hidden_size, output_size):
        """Constructor for the Discriminator
        Args:
            input: is the dimensionality of the input sample
            hidden_size: is the number of hidden units in the GAN
            output_size: is the dimensionality of the output sample"""
        super(Discriminator, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.main = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size),
            nn.Sigmoid(),
            nn.Linear(self.hidden_size, self.output_size),
            nn.Sigmoid(),
            nn.Linear(self.output_size, self.output_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

In [85]:
"""Task 2. Sample Z"""


def sample_Z(mu, sigma, sampleType):
    """Creates input for the generator and discriminator
    Args:
        mu: is the mean of the normal distribution
        sigma: is the standard deviation of the normal distribution
        sampleType: is the sample type G or D
    Returns:
        Z: a torch.tensor containing the input for the generator or
        discriminator"""

    return realSample_Z(mu, sigma, sampleType, (1, 50))


def realSample_Z(mu, sigma, sampleType, size):
    """Does the actual work since we need size"""

    if sampleType == 'G':
        Z = torch.normal(mu, sigma, size=size)
    elif sampleType == 'D':
        # size = (1, 1)
        Z = torch.randn(size) * sigma + mu
    else:
        return 0
    return Z

In [120]:
"""Task 3. Train Discriminator"""


def train_dis(Gen, Dis, dInputSize, gInputSize, mbatchSize, steps, optimizer,
              crit):
    """Trains the discriminator
    Args:
        Gen: is the generator model
        Dis: is the discriminator model
        dInputSize: is the dimensionality of the discriminator input
        gInputSize: is the dimensionality of the generator input
        mbatchSize: is the batch size
        steps: is the number of discriminator iterations
        optimizer: is an stochastic gradient descent optimizer object
        crit: is the discriminator BCEloss function
    Returns:
        lossFake: the discriminator loss
        lossReal: the discriminator loss
        FakeData: the generated sample
        RealData: the real sample"""


    for _ in range(steps):
        # resetting the gradients for each iteration
        Dis.zero_grad()

        # Generate noise and fake data labels which is confusing since it is
        # one number??? Randomly throwing one sample at a time until it thinks
        # it was a truly random sample??
        # edit: I think we throw it one real one for shits and giggles
        noise = realSample_Z(0, 1, 'D', (mbatchSize, dInputSize))
        fake_data_labels = torch.ones(mbatchSize, 1)

        # Generate fake data using the Generator on a decent sample size and
        # matching labels
        fakers = realSample_Z(0, 1, 'G', (mbatchSize, gInputSize))
        fakers_output = Gen(fakers)
        fakers_output_labels = torch.zeros(mbatchSize, 1)

        # bringing data and labels together
        data = torch.cat((noise, fakers_output))
        data_labels = torch.cat((fake_data_labels, fakers_output_labels))

        # Train the discriminator by feeding it the data and then calculating
        # loss and backpropagating
        trained_data = Dis(data)
        loss = crit(trained_data, data_labels)
        loss.backward()
        optimizer.step()
    optimizer.zero_grad()
    return crit(trained_data, data_labels), data


In [130]:
disTrain = train_dis

learning_rate = 1e-3
sgd_momentum = 0.9
g_input_size = 1
g_hidden_size = 5
g_output_size = 1
minibatch_size =  500
d_input_size = 1
d_hidden_size = 10
d_output_size = 1
num_epochs = 50
d_steps = 20
g_steps = 20

G_Test = Generator(1,5,1)
D_Test = Discriminator(1,10,1)



opt_Test = optim.SGD(G_Test.parameters(), lr=learning_rate, momentum=sgd_momentum)
crit_Test = nn.BCELoss()

print(disTrain(G_Test,D_Test, d_input_size,g_input_size, minibatch_size, d_steps, opt_Test, crit_Test))

(tensor(0.7306, grad_fn=<BinaryCrossEntropyBackward0>), tensor([[-1.5095e+00],
        [-9.5188e-01],
        [-1.2769e+00],
        [-1.2148e+00],
        [-3.7235e-01],
        [-1.6630e+00],
        [ 3.7231e-01],
        [-8.5966e-01],
        [-1.1924e+00],
        [ 1.5074e+00],
        [-9.5151e-02],
        [ 1.3188e-01],
        [ 5.4798e-01],
        [-8.9247e-01],
        [ 2.2538e+00],
        [ 1.8908e+00],
        [ 4.1521e-01],
        [ 2.9152e-01],
        [-2.0931e+00],
        [ 8.5580e-01],
        [-6.2923e-01],
        [ 7.8235e-01],
        [-1.7030e+00],
        [ 2.1452e-01],
        [-1.1905e+00],
        [-8.7436e-01],
        [ 5.5401e-01],
        [ 1.2892e+00],
        [-1.6570e-01],
        [ 1.8773e+00],
        [ 4.0546e-01],
        [-5.4835e-01],
        [-1.4088e+00],
        [-4.5905e-01],
        [ 5.8086e-01],
        [ 1.1054e+00],
        [-5.2797e-01],
        [ 1.6301e+00],
        [ 1.0056e+00],
        [ 1.2944e-01],
        [ 1.1254e+00],
 

In [156]:
"""Task 4. Train Generator"""


def train_gen(Gen,Dis, gInputSize, mbatchSize, steps, optimizer, crit):
    """Trains the generator
    Args:
        Gen: is the generator model
        Dis: is the discriminator model
        gInputSize: is the dimensionality of the generator input
        mbatchSize: is the batch size
        steps: is the number of generator iterations
        optimizer: is an stochastic gradient descent optimizer object
        crit: is the discriminator BCEloss function
    Returns:
        error of fake data set
        fake data set of type torch.tensor"""

    for _ in range(steps):

        # Generate fake data
        fakers = realSample_Z(0, 1, 'G', (mbatchSize, gInputSize))
        fakers_output = Gen(fakers)
        labels = torch.ones((mbatchSize, 1))

        # Train the generator
        # resetting the gradients for each iteration
        Gen.zero_grad()
        trained_data = Dis(fakers_output)
        loss = crit(trained_data, labels)
        loss.backward()
        optimizer.step()

    return loss, fakers

In [159]:
genTrain = train_gen

learning_rate = 1e-3
sgd_momentum = 0.9
g_input_size = 1   
g_hidden_size = 5  
g_output_size = 1  
minibatch_size =  500 
d_input_size = 4 
d_hidden_size = 10 
d_output_size = 1  
num_epochs = 50
d_steps = 20
g_steps = 20



G_Test = Generator(1,5,1)
D_Test = Discriminator(1,10,1)



dOpt_Test = optim.SGD(D_Test.parameters(), lr=learning_rate, momentum=sgd_momentum)
crit_Test = nn.BCELoss()  


print(genTrain(G_Test,D_Test,g_input_size, minibatch_size, g_steps, dOpt_Test, crit_Test))

(tensor(0.8192, grad_fn=<BinaryCrossEntropyBackward0>), tensor([[ 0.0649],
        [ 0.2725],
        [-0.7063],
        [-0.0910],
        [ 0.3962],
        [-1.0028],
        [-0.6839],
        [-1.2459],
        [-1.6628],
        [ 0.2825],
        [ 2.2248],
        [ 0.9651],
        [ 0.4878],
        [ 1.0936],
        [ 0.9315],
        [ 0.5723],
        [-1.3034],
        [-0.8305],
        [-0.2251],
        [-0.2505],
        [ 1.3624],
        [-1.6117],
        [ 0.6064],
        [ 0.1960],
        [ 0.8788],
        [ 0.0636],
        [-0.5797],
        [ 0.7729],
        [-0.7632],
        [ 0.5737],
        [ 1.6982],
        [-0.3649],
        [-0.4147],
        [-0.9624],
        [ 0.2423],
        [-0.9444],
        [ 2.0783],
        [-1.3376],
        [ 0.0744],
        [ 0.5820],
        [-0.5863],
        [ 0.2035],
        [ 0.4596],
        [-0.5962],
        [-0.4363],
        [ 1.2100],
        [ 0.2786],
        [ 1.1968],
        [-2.0275],
        [ 1.7